# Work In Progress (WIP)

# 1. The Qubit

In [1]:
#  Ensures you have dependencies installed in your Kaggle Notebooks environment
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -U fastcore
    !pip install -U plotly

In [2]:
import numpy as np
import plotly.graph_objects as go

## What is a qubit?

The fundamental unit of classical computing is the bit, which can be 0 or 1. These bits can be represented in a variety of ways. We can turn an electric current on (1) or off (0). We can take a coin and say heads is 1 and tails is 0. We even keep track of 0 and 1 on a piece of paper. The same is true for a qubit, but the qubit can also have any state between 0 and 1. This often leads to misleading statement like "a qubit can be 0 and 1 at the same time" and "a qubit can hold an infinite amount of information". This has some philosophical validity for the quantum state, but we cannot use it in the same way as classical bits. This is because to obtain information from the quantum state, it first has to be measured and this measuring process collapses the qubit to a (classical) 0 or 1. We will discuss measurement in more detail later.

Like with classical bits, there are also a variety of ways to represent a qubit. Examples of ways people have implemented qubits are:

1. Laser pulses
2. Electrons
3. Superconductors

While using different physical representations have some different operational implications, the mathematical representation of qubits stays exactly the same. Compare it to classical computer science where the same algorithms hold for all computer architectures. Computer scientists don't have to develop sorting algorithms from scratch every time a new chip is released.

We will focus on the fundamental representation of the qubit as a vector of numbers. For an intuitive video explanation of a qubit, see [this video](https://www.youtube.com/watch?v=kgSVkVNxXyU).


Quantum computing is all about a state evolving over time through a series of operations (logic gates). In general all algorithms start from a "zero state" or "basis state". For practical purposes this state is equivalent to a 0 in classical computing. For one qubit, the basis state is represented as:

$$
\begin{bmatrix}
1 \\
0
\end{bmatrix} = |0\rangle
$$

The numbers in this vector are complex numbers, but these are often omitted for simplicity. We will go more in-depth in the complex part further in the notebook. The $|0\rangle$ representation is used as short-hand notation and is called [Bra-ket notation](https://en.wikipedia.org/wiki/Bra%E2%80%93ket_notation). The state $|0\rangle$ is called a "Ket". Quantum physicists have a habit of using lots of short-hand notation to save space. We will use the Ket notation, but always show you the full representation in code.

In [6]:
zero_state = np.array([[1, 0]], dtype=complex).T
# Zero state "Ket"
zero_state

array([[1.+0.j],
       [0.+0.j]])

A Ket can be converted into a "Bra" state by transposing it and flipping the sign of the complex part. This is called a ["conjugate transpose"](https://en.wikipedia.org/wiki/Complex_conjugate). The symbol for this operation is $\dagger$ (dagger).

$$
\begin{bmatrix}
1 \\
0
\end{bmatrix}^\dagger =
\begin{bmatrix}
1 & 0
\end{bmatrix} = \langle 0 |
$$

In [10]:
# Zero state "Bra"
zero_state.conj().T

array([[1.-0.j, 0.-0.j]])

The opposite of the basis state is the one state:

$$
\begin{bmatrix}
0 \\
1
\end{bmatrix} = |1\rangle
$$


In [12]:
one_state = np.array([[0, 1]], dtype=complex).T
one_state

array([[0.+0.j],
       [1.+0.j]])

So how do we get from a $|0\rangle$ to a $|1\rangle$ in quantum computing? This is achieved through quantum logic **gates**. One of the simplest meaningful quantum logic gates is the X (NOT) gate. A quantum logic gate for a single qubit is a $2 \times 2$ complex matrix. The X gate flips the state of the qubit.

$$
X = \begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}
$$



In [13]:
X = np.array([[0, 1], 
              [1, 0]], dtype=complex)
X @ zero_state

array([[0.+0.j],
       [1.+0.j]])

$$
X |0\rangle = |1\rangle
$$


This vector-matrix multiplication of a logic gate and a state, although very simple, is already a quantum computation! We're doing it! Applying the X gate again on our obtained $|1\rangle$ state will return us to the $|0\rangle$ state:

$$
X(X |0\rangle) = X |1\rangle = |0\rangle
$$


There are many different quantum logic gates and you can make your own as long as they pass some requirements. We will go in-depth on all kinds of logic gates in the next notebook.

## Requirements for a qubit

Thus far our quantum computations have not been more powerful than performing a classical bit flips. That changes now with superposition. 

The main requirement for a valid qubit state is that the square of the absolute value of the two numbers in the vector add up to 1. In other words, for a vector $\begin{bmatrix} a \\ b \end{bmatrix}$ to be a valid qubit state, it must be true that $|a|^2 + |b|^2 = 1$. This is because the statevector describes a probability distribution. Therefore, we can create a state where we have an equal probability of being in the $|0\rangle$ or $|1\rangle$ state.

$$
\begin{bmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{bmatrix} = \begin{bmatrix} 0.707 \\ 0.707 \end{bmatrix}  = \frac{1}{\sqrt{2}} |0\rangle + \frac{1}{\sqrt{2}} |1\rangle
$$

This state is called a perfect superposition. Perfect randomness like this cannot be achieved in classical computing and definitely not with a single bit. Classical computers can only achieve [pseudo-randomness](https://en.wikipedia.org/wiki/Pseudorandomness), while a qubit can achieve true randomness.


In [ ]:
superposition_state = np.array([1/np.sqrt(2), 1/np.sqrt(2)], dtype=complex)
superposition_state


What happens if we apply our X gate on this superposition state.

In [ ]:
X @ superposition_state

Interesting! Nothing changed. This is because we are trying to flip around an axis that is perpendicular to the superposition state. We are trying to flip around the X axis, while the state is in the middle. Also recall that the statevector of a qubit contains two complex numbers. The state of a qubit therefore envelops a 3-dimensional space and all quantum operations can be represented as rotations in this space. Let's look at a qubit visually with [plotly](https://plotly.com) to fully grasp the significance of this. A visual representation of a qubit is called a [Bloch sphere](https://en.wikipedia.org/wiki/Bloch_sphere).

In [36]:
def bloch(state: np.array):
    """ Plot single qubit state on a Bloch sphere. """
    def calculate_coordinates(theta, phi):
        x = np.sin(theta) * np.cos(phi)
        y = np.sin(theta) * np.sin(phi)
        z = np.cos(theta)
        return x, y, z

    fig = go.Figure()
    alpha, beta = state[0], state[1]
    theta = 2 * np.arccos(np.abs(alpha))
    phi = np.angle(beta) - np.angle(alpha)
    x, y, z = calculate_coordinates(theta, phi)
    surface_phi, surface_theta = np.mgrid[0:2*np.pi:100j, 0:np.pi:50j]
    xs, ys, zs = calculate_coordinates(surface_theta, surface_phi)

    fig.add_trace(go.Surface(x=xs, y=ys, z=zs, opacity=0.5, colorscale='Blues', showscale=False))

    fig.add_trace(go.Scatter3d(x=[0, x],y=[0, y],z=[0, z], mode='lines+markers+text', marker=dict(size=10, color='green'),
        line=dict(color='green', width=8), textposition="top center", showlegend=True,name=f"{alpha:.3f}|0⟩ + {beta:.3f}|1⟩"))

    fig.add_trace(go.Scatter3d(x=[0, 0, 1, -1, 0, 0],y=[0, 0, 0, 0, 1, -1], z=[1, -1, 0, 0, 0, 0],
        mode='markers', marker=dict(size=5, color='black'), hovertext=['|0⟩', '|1⟩', '|+⟩', '|-⟩', '|i⟩', '|-i⟩'],
        showlegend=False, name="Basis states"))

    boundary_phi = np.linspace(0, 2 * np.pi, 100)
    coords = [(np.cos(boundary_phi), np.sin(boundary_phi), np.zeros_like(boundary_phi)),
              (np.zeros_like(boundary_phi), np.cos(boundary_phi), np.sin(boundary_phi)),
              (np.cos(boundary_phi), np.zeros_like(boundary_phi), np.sin(boundary_phi)) ]
    
    for x, y, z in coords:
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='black', width=2), showlegend=False, name="Axes"))

    fig.update_layout(legend=dict( font=dict(size=20), x=0.05,y=0.95, xanchor='left', yanchor='top', bgcolor='rgba(0,0,0,0)',),
                      margin=dict(l=0, r=0, t=0, b=0))
    return fig

# TODO: Explain state (north pole, south pole) and why X on superposition state does not change anything.

In [ ]:
bloch(zero_state)

In [ ]:
bloch(one_state)

In [ ]:
bloch(superposition_state)

# TODO Explain Hadamard (Zero state to superposition state)

## Measurement

Measurement collapses the qubit to a classical state. Measurements can be made in several bases.

# BONUS: Qutrit and beyond

# Work In Progress (WIP)
